# Introduction

Prompt engineering is a concept in Natural Language Processing (NLP) that involves embedding descriptions of tasks in input to prompt an AI model to output the desired results.  A prompt typically includes the problem description, instructions on how to solve the problem, and examples of correct problem and solution pairs.

This notebook provides a few samples of prompts used to accomplish tasks like, text generation, summarization, and classification using the Azure OpenAI Service.

## Setup

Configuration for using an Azure OpenAI endpoint.  You must first have setup you service and deployed a gpt-35-turbo model.  For instructions to set this up, please see [Resource creation & model deployment](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and more information about [GPT-35-Turbo & GPT-4 models] (https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions).

These prompts should work with other model families as well, but has only been tested with that type, and you may see different results to the prompts shown.

In [1]:
#!pip install openai

In [2]:
import openai
import os
import json

### Parameter Configuration
Edit (or create) the config.json file to associate with this notebook containing the information for the Azure OpenAI endpoint you created.


```
{
    "OPENAI_API_BASE":"https://<Your Azure Resource Name>.openai.azure.com",
    "OPENAI_API_KEY":"<OpenAI API Key>",
    "OPENAI_API_VERSION":"<OpenAI API Version>",
    "GPT_MODEL":"<GPT Model Deployment Name>"
}
```

**NOTES:** 
* Access the OPENAI_API_KEY value from the Azure Portal. Go to https://portal.azure.com, find your resource and then under "Resource Management" -> "Keys and Endpoints" look for one of the "Keys" values.

* This notebook uses the ChatCompletion method of the [OpenAI Python Library](https://github.com/openai/openai-python#chat-completions) -  the versions supported for Azure OpenAI are listed in [Chat completions API Reference](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions).  The "2023-05-15" version was used for this notebook.

The next step loads the configurations from config.json file to setup openai_api_base, openai_api_key, openai_api_version, type, and model deployment within the notebook.

In [3]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base = config_details['OPENAI_API_BASE']

# The API key for your Azure OpenAI resource.
openai_api_key = config_details["OPENAI_API_KEY"]

# See Notes above for version options (format is YYYY-MM-DD)
openai_api_version = config_details['OPENAI_API_VERSION']

# The model deployment name from OpenAI service.
deployment_name = config_details['GPT_MODEL']

openai.api_base = openai_api_base
openai.api_key = openai_api_key
openai.api_version = openai_api_version
openai.api_type = "azure"

### AOAI API Call

This function is used throughout the notebook to manage requests to the Azure OpenAI service.

In [4]:
# manages call to API to get a ChatCompletion for a text prompt/message.
def aoai_completion(user_request=None, messages=None, model=deployment_name, temperature=0):

    if messages == None:
        messages = [{"role": "user", "content": user_request}]

    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature, 
    )

    return response, response.choices[0].message["content"]

## Simple Prompt

In [ ]:
# Example prompt to check connectivity
prompt = "Hello, how are you?"

aoai_response, completion = aoai_completion(prompt)
print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')

Customer: Hello, how are you?

Assistant Response: As an AI language model, I don't have feelings, but I'm functioning well. How can I assist you today?

Prompt tokens: 14
Completion tokens: 25
Total tokens: 39


## Text Classification 

### Named Entity Recognition (Zero-Shot)

In [ ]:
address = """
Dear Kelly,
It was great to talk to you at the seminar. I thought Jane's talk was quite good.
Thank you for the book. Here's my address 2111 Ash Lane, Crestview CA 92002
Best,
Maya
"""


In [ ]:
prompt = f"""
Extract the name and mailing address from this email:

Mailing Address: ```{address}```

"""
aoai_response, completion = aoai_completion(prompt)
print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')

## Summarization

### Semantic Summarization

Generates new text using natural language generation techniques that represents the most relevant/important information.

In [7]:
recipe = """
Making sourdough bread is a bit different from making regular bread because it requires a sourdough starter. 
To make the starter, you'll need flour and water. Mix equal parts of flour and water in a jar and leave it on your 
counter. After a few days, the mixture will start to bubble and ferment, which is when the sourdough starter is ready 
to use.\n\nOnce you have the starter, you'll need to mix it with more flour and water to make the bread dough. 
In a mixing bowl, add the flour, salt, and a portion of the sourdough starter. Slowly mix in water until the dough 
comes together and is slightly sticky.\n\nKnead the dough for about 10 minutes on a floured surface until 
it's smooth and elastic. Place the dough in a bowl and cover it with a damp cloth. Let it rise in a warm place for 
several hours or until it has doubled in size.\n\nAfter it has risen, gently shape the dough into a round or oblong 
shape. Place it in a proofing basket or a greased bread pan and cover it with a damp cloth. Let it rise for another 
few hours or until it has doubled in size again.\n\nPreheat your oven to 425°F and place a baking dish with water 
in the oven. This will create steam, which will help the bread rise and develop a crispy crust. Score the top of 
the bread with a sharp knife or razor blade.\n\nBake the bread in the oven for 40-45 minutes or until the crust 
is golden brown. Remove from the oven and let it cool completely before slicing. Sourdough bread has a tangy 
flavor and chewy texture that's perfect for sandwiches or toast.
"""

In [8]:
prompt = f"""
Generate a short summary of the recipe below, delimited by three
backticks, in at most 50 words. 

Recipe: ```{recipe}```
"""

aoai_response, completion = aoai_completion(prompt)
print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')

Customer: 
Generate a short summary of the recipe below, delimited by three
backticks, in at most 50 words. 

Recipe: ```
Making sourdough bread is a bit different from making regular bread because it requires a sourdough starter. 
To make the starter, you'll need flour and water. Mix equal parts of flour and water in a jar and leave it on your 
counter. After a few days, the mixture will start to bubble and ferment, which is when the sourdough starter is ready 
to use.

Once you have the starter, you'll need to mix it with more flour and water to make the bread dough. 
In a mixing bowl, add the flour, salt, and a portion of the sourdough starter. Slowly mix in water until the dough 
comes together and is slightly sticky.

Knead the dough for about 10 minutes on a floured surface until 
it's smooth and elastic. Place the dough in a bowl and cover it with a damp cloth. Let it rise in a warm place for 
several hours or until it has doubled in size.

After it has risen, gently shape the d

In [9]:
print(len(completion.split(" ")), ' words')
print(len(completion), ' characters')

51  words
277  characters


Note, that the number of words/characters is known not to be exact with the GPT-35-Turbo model, but should be close.

In [10]:
prompt = f"""
Summarize the recipe below, delimited by three backticks, but focus on the sourdough instructions.

Recipe: ```{recipe}```
"""

aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: The recipe explains how to make sourdough bread, which requires a sourdough starter made from equal parts flour and water left to ferment. The starter is then mixed with flour, salt, and water to make the bread dough, which is kneaded and left to rise. The dough is shaped and left to rise again before being baked in the oven with steam to create a crispy crust. Sourdough bread has a tangy flavor and chewy texture.

Prompt tokens: 400
Completion tokens: 93
Total tokens: 493


### Extractive Summarization 

Extrative summarization selects parts of the original text to form a summary.

In [11]:
prompt = f"""
Extract only the sentences relevant to the sourdough information from the recipe below, delimited by three backticks. 

Recipe: ```{recipe}```
"""

aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: To make the starter, you'll need flour and water. Mix equal parts of flour and water in a jar and leave it on your 
counter. After a few days, the mixture will start to bubble and ferment, which is when the sourdough starter is ready 
to use.

Once you have the starter, you'll need to mix it with more flour and water to make the bread dough. 

Sourdough bread has a tangy flavor and chewy texture that's perfect for sandwiches or toast.

Prompt tokens: 401
Completion tokens: 103
Total tokens: 504


### Summarize Multiple Recipes

In [12]:
recipe2 ="""
    Ingredients:

    1 3/4 cups all-purpose flour
    2 cups granulated sugar
    3/4 cup unsweetened cocoa powder
    2 tsp baking soda
    1 tsp baking powder
    1 tsp salt
    1 cup buttermilk
    1/2 cup vegetable oil
    2 large eggs
    2 tsp vanilla extract
    1 cup boiling water

    Instructions:
    Preheat your oven to 350°F. Grease two 9-inch round cake pans with butter or cooking spray and line the bottom with parchment paper.
    In a large mixing bowl, whisk together the flour, sugar, cocoa powder, baking soda, baking powder, and salt until well combined.
    In a separate bowl, whisk together the buttermilk, vegetable oil, eggs, and vanilla extract until smooth.
    Add the wet ingredients to the dry ingredients and mix until well combined.
    Slowly pour in the boiling water and mix until the batter is smooth.
    Pour the batter evenly into the prepared cake pans.
    Bake for 30-35 minutes or until a toothpick inserted into the center of the cakes comes out clean.
    Let the cakes cool in the pans for 10 minutes before removing them and placing them on a wire rack to cool completely.
    Once the cakes are cool, you can frost them with your favorite frosting or serve them as is.

    Enjoy your delicious homemade chocolate cake!

"""

recipe3 ="""
    Ingredients:

    1 standing rib roast (4-5 pounds)
    2 tbsp olive oil
    2 tbsp kosher salt
    1 tbsp black pepper
    1 tbsp garlic powder
    1 tbsp dried thyme
    1 tbsp dried rosemary

    Instructions:
    Preheat your oven to 450°F.
    Rub the olive oil all over the standing rib roast.
    In a small mixing bowl, combine the kosher salt, black pepper, garlic powder, dried thyme, and dried rosemary. Mix well.
    Sprinkle the spice mixture all over the standing rib roast, making sure to coat it evenly.
    Place the standing rib roast on a roasting pan with the bone side down.
    Roast in the oven for 15 minutes.
    Reduce the oven temperature to 325°F and continue roasting for 1-2 hours or until the internal temperature reaches 135°F for medium-rare or 145°F for medium.
    Remove the standing rib roast from the oven and let it rest for 10-15 minutes before slicing and serving.

    Enjoy your delicious and flavorful standing rib roast!
"""


In [13]:
recipes = [recipe, recipe2, recipe3]

for i in range(len(recipes)):
    prompt = f"""
    Generate a short summary of the recipe below, delimited by three backticks in at most 20 words. 

    Recipe: ```{recipes[i]}```
    """

    aoai_response, completion = aoai_completion(prompt)
    print(i, completion, "\n")

0 This recipe explains how to make sourdough bread using a sourdough starter, flour, water, and salt. The dough is kneaded, left to rise, shaped, and baked in the oven with steam to create a crispy crust. 

1 A classic chocolate cake recipe made with flour, sugar, cocoa powder, buttermilk, eggs, and vegetable oil, topped with frosting. 

2 A flavorful standing rib roast recipe with olive oil, spices, and herbs, roasted in the oven for 1-2 hours. 



## Inferencing

Inferencing is the process of using a model to make predictions on new data - in the context of the LLM, it refers to the ability to understand user input and generate appropriate responses.  Some ways this could be used is for understanding sentiment, extracting components of the text, and classification.

### Sentiment / Emotion

In [14]:
pos_review = """
    I organized a surprise birthday celebration at Joe's Restaurant and it was fantastic. The food 
    was delicious and beautifully presented, with a great selection of options for all tastes and 
    dietary restrictions. The staff were friendly and accommodating, making sure we had everything we 
    needed throughout the night. The atmosphere was cozy and intimate, perfect for a celebration with 
    friends and family. I highly recommend Joe's Restaurant for anyone looking for a great dining 
    experience. Thank you to the staff for making our celebration so memorable!
"""

In [15]:
neg_review = """
    I was really disappointed with my experience at this restaurant. The service was slow and inattentive, 
    and the food was bland and overpriced. I ordered a steak that was supposed to be cooked medium-rare, 
    but it came out well-done and tough. When I brought it to the attention of the server, they didn't seem to care 
    and didn't offer to fix it or make it right. To top it off, the atmosphere was noisy and chaotic, 
    which made it difficult to have a conversation. Overall, I wouldn't recommend this restaurant and 
    won't be returning.
"""

In [16]:
prompt = f"""
What is the sentiment of the following restaurant review? Give your answer as a single word, either "positive" or "negative".

Review text: '''{pos_review}'''
"""
aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: positive

Prompt tokens: 148
Completion tokens: 1
Total tokens: 149


In [17]:
prompt = f"""
What is the sentiment of the following restaurant review? Give your answer as a single word, either "positive" or "negative".

Review text: '''{neg_review}'''
"""

aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: negative

Prompt tokens: 169
Completion tokens: 1
Total tokens: 170


In [60]:
prompt = f"""
List 3 emotions that the writer of the restaurant review is expressing. 
The review is delimited by three backticks. 
Format the list as lower-case words separated by commas.

Review text: '''{pos_review}'''
"""

aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: happy, satisfied, grateful

Prompt tokens: 158
Completion tokens: 5
Total tokens: 163


In [18]:
prompt = f"""
Is the writer of the following review expressing anger?
The review is delimited by three backticks. 
Give your answer as either angry or not angry.

Review text: '''{neg_review}'''
"""
aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: angry

Prompt tokens: 173
Completion tokens: 2
Total tokens: 175


In [19]:
prompt = f"""
Is the writer of the following review expressing anger?
The review is delimited with triple backticks. 
Give your answer as either angry or not angry.

Review text: '''{pos_review}'''
"""
aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: not angry

Prompt tokens: 152
Completion tokens: 2
Total tokens: 154


### Topic Extraction

In [20]:
prompt = f"""
Determine five topics that are being discussed in the 
following text sample, which is delimited by three backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{pos_review}'''
"""
aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: Surprise birthday celebration, Joe's Restaurant, Food, Staff, Dining experience.

Prompt tokens: 167
Completion tokens: 16
Total tokens: 183


In [21]:
prompt = f"""
Identify the following items from the review text: 
- Name of the restaurant
- Favorite part of experience
- Sentiment (positive or negative) of the Reviewer

The review is delimited with triple backticks. 
Format your response as a list including  
"Topic" and "Extracted" as the key. 
If the information isn't present, use "unknown" as the value. 
Make your response as short as possible.
  
Review text: '''{pos_review}'''
"""
aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: - Topic: Name of the restaurant
- Extracted: Joe's Restaurant

- Topic: Favorite part of experience
- Extracted: The food was delicious and beautifully presented, with a great selection of options for all tastes and dietary restrictions.

- Topic: Sentiment of the Reviewer
- Extracted: Positive

Prompt tokens: 214
Completion tokens: 64
Total tokens: 278


### Classification

In [22]:
news_headlines = """
"Record Heatwave Sweeps Across the West"
"New Study Shows Benefits of Plant-Based Diets"
"AI-Powered Drone Revolutionizes Agriculture Industry"
"LeBron James Signs Four-Year Deal with Lakers"
"NASA Launches Mission to Explore Europa's Ocean"
"""

In [23]:
prompt = f"""
Classify the following news headlines into 1 of the following categories: sports, news, food, technology, weather

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{news_headlines}'''
"""
aoai_response, completion = aoai_completion(prompt)
#print(f"Customer: {prompt}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')


Assistant Response: weather, food, technology, sports, technology

Prompt tokens: 113
Completion tokens: 9
Total tokens: 122


## Chatting / Advanced Conversations

In the previous steps using the "get_completion" function, the message was constructed using a default pattern supplying just the prompt:

```
messages = [{"role": "user", "content": prompt}]
```

The same API call/SDK method is used for conversations, we add more context and information to each call by expanding the roles used to help manage the conversation and ensure that the AI model responds appropriately to requests and stays within its intended scope and does not provide inappropriate or irrelevant responses.

The **system** role handles the interactions between the user and the assistant.  You can provide details about what is expected during their expectations by using the "system" role.

The **user** initiates the conversation.

The **assistant** is the AI model responding to user requests and messages.

The format for adding the additional information is:

```
messages = [
	{"role": "system", "content": "<instructions or description of the environment>"},
	{"role": "user", "content": "<user request>" },
	{"role": "assistant", "content": "<response to the user request>" }
]
```

Using these additional roles allow for you to provide details of how the assistant should respond, and to provide one/few-shot examples for the conversation.

In [24]:
messages =  [  
{'role':'system', 'content': 'You are an assistant that speaks like a Pirate.'},    
{'role':'user', 'content': 'Tell me a joke'},   
{'role':'assistant', 'content': 'Why did the tomato turn red?'},   
{'role':'user', 'content':'I don\'t know.'}  ]

In [25]:
aoai_response, completion = aoai_completion(user_request = None, messages = messages)
print(f"Customer: {messages}")
print(f"\nAssistant Response: {completion}")
print(f'\nPrompt tokens: {aoai_response["usage"]["prompt_tokens"]}')
print(f'Completion tokens: {aoai_response["usage"]["completion_tokens"]}')
print(f'Total tokens: {aoai_response["usage"]["total_tokens"]}')

Customer: [{'role': 'system', 'content': 'You are an assistant that speaks like a Pirate.'}, {'role': 'user', 'content': 'Tell me a joke'}, {'role': 'assistant', 'content': 'Why did the tomato turn red?'}, {'role': 'user', 'content': "I don't know."}]

Assistant Response: Because it saw the salad dressing! Ahoy!

Prompt tokens: 49
Completion tokens: 10
Total tokens: 59
